In [27]:
!pip install pymongo
!pip install faker
!pip install unidecode
!pip install pandas
!pip install openpyxl
!pip install apyori
!pip install ipywidgets
!pip install prophet

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Using cached prophet-1.1.5-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.5 kB)
  Using cached cmdstanpy-1.2.2-py3-none-any.whl.metadata (4.1 kB)
  Using cached holidays-0.48-py3-none-any.whl.metadata (23 kB)
  Using cached tqdm-4.66.4-py3-none-any.whl.metadata (57 kB)
  Using cached importlib_resources-6.4.0-py3-none-any.whl.metadata (3.9 kB)
  Using

In [2]:
from pymongo import MongoClient
from datetime import datetime, timedelta
import pandas as pd
import random

### MongoDB

In [3]:
uri = 'mongodb://localhost/'
db = MongoClient(uri)['vendas']

### estoque

In [49]:
xls = pd.read_excel('estoque.xlsx')

In [50]:
estoques = []
for i, e in enumerate(xls.to_dict('records')):
  estoques.append({
    'id': i + 1,
    'produto': {
      'marca': {
        'nome': e['marca'],
      },
      'categoria': {
        'titulo': e['categoria'],
      },
      'descricao': e['descricao'],
    },
    'quantidade': e['quantidade'],
    'preco_compra': e['preco_compra'],
    'preco_venda': e['preco_compra'] * 2,
  })
estoques

[{'id': 1,
  'produto': {'marca': {'nome': 'KazeKami'},
   'categoria': {'titulo': 'Secadores'},
   'descricao': 'Secador de Parede 1200W'},
  'quantidade': 1000,
  'preco_compra': 149.73,
  'preco_venda': 299.46},
 {'id': 2,
  'produto': {'marca': {'nome': 'KazeKami'},
   'categoria': {'titulo': 'Secadores'},
   'descricao': 'Secador de Parede 1200W'},
  'quantidade': 1000,
  'preco_compra': 169.9,
  'preco_venda': 339.8},
 {'id': 3,
  'produto': {'marca': {'nome': 'KazeKami'},
   'categoria': {'titulo': 'Secadores'},
   'descricao': 'Secador de Parede 1400W'},
  'quantidade': 1000,
  'preco_compra': 460.35,
  'preco_venda': 920.7},
 {'id': 4,
  'produto': {'marca': {'nome': 'KazeKami'},
   'categoria': {'titulo': 'Secadores'},
   'descricao': 'Secador de Parede 1400W (Preto)'},
  'quantidade': 1000,
  'preco_compra': 469.26,
  'preco_venda': 938.52},
 {'id': 5,
  'produto': {'marca': {'nome': 'KazeKami'},
   'categoria': {'titulo': 'Secadores'},
   'descricao': 'Secador de Parede 150

In [51]:
db['venda_estoque'].delete_many({})
db['venda_estoque'].insert_many(estoques)

InsertManyResult([ObjectId('6642bf3dfa8ceb442718d263'), ObjectId('6642bf3dfa8ceb442718d264'), ObjectId('6642bf3dfa8ceb442718d265'), ObjectId('6642bf3dfa8ceb442718d266'), ObjectId('6642bf3dfa8ceb442718d267'), ObjectId('6642bf3dfa8ceb442718d268'), ObjectId('6642bf3dfa8ceb442718d269'), ObjectId('6642bf3dfa8ceb442718d26a'), ObjectId('6642bf3dfa8ceb442718d26b'), ObjectId('6642bf3dfa8ceb442718d26c'), ObjectId('6642bf3dfa8ceb442718d26d'), ObjectId('6642bf3dfa8ceb442718d26e'), ObjectId('6642bf3dfa8ceb442718d26f'), ObjectId('6642bf3dfa8ceb442718d270'), ObjectId('6642bf3dfa8ceb442718d271'), ObjectId('6642bf3dfa8ceb442718d272'), ObjectId('6642bf3dfa8ceb442718d273'), ObjectId('6642bf3dfa8ceb442718d274'), ObjectId('6642bf3dfa8ceb442718d275'), ObjectId('6642bf3dfa8ceb442718d276'), ObjectId('6642bf3dfa8ceb442718d277'), ObjectId('6642bf3dfa8ceb442718d278'), ObjectId('6642bf3dfa8ceb442718d279'), ObjectId('6642bf3dfa8ceb442718d27a'), ObjectId('6642bf3dfa8ceb442718d27b'), ObjectId('6642bf3dfa8ceb442718d2

# Pedidos

### base original

In [4]:
df_ori = pd.read_csv('Sales_Product_Combined.csv')
df_ori

,Order ID,Product,Quantity Ordered,Price,Order Date,Time,Purchase Address,City,Product Type
0,176558,USB-C Charging Cable,2,11.95,19-04-2019,8:46 AM,"917 1st St, Dallas, TX 75001",Dallas,Cable
1,176559,Bose SoundSport Headphones,1,99.99,07-04-2019,10:30 PM,"682 Chestnut St, Boston, MA 02215",Boston,Headphones
2,176560,Google Phone,1,600,12-04-2019,2:38 PM,"669 Spruce St, Los Angeles, CA 90001",Los Angeles,Phone
3,176560,Wired Headphones,1,11.99,12-04-2019,2:38 PM,"669 Spruce St, Los Angeles, CA 90001",Los Angeles,Headphones
4,176561,Wired Headphones,1,11.99,30-04-2019,9:27 AM,"333 8th St, Los Angeles, CA 90001",Los Angeles,Headphones
...,...,...,...,...,...,...,...,...,...
185945,259353,AAA Batteries (4-pack),3,2.99,17-09-2019,8:56 PM,"840 Highland St, Los Angeles, CA 90001",Los Angeles,Batteries
185946,259354,iPhone,1,700,01-09-2019,4:00 PM,"216 Dogwood St, San Francisco, CA 94016",San Francisco,iPhone
185947,259355,iPhone,1,700,23-09-2019,7:39 AM,"220 12th St, San Francisco, CA 94016",San Francisco,iPhone
185948,259356,34in Ultrawide Monitor,1,379.99,19-09-2019,5:30 PM,"511 Forest St, San Francisco, CA 94016",San Francisco,Monitor


In [5]:
# quantidade por cidade
df_ori.groupby('City')['Quantity Ordered'].count().sort_values()

City
Austin            9905
Portland         12465
Seattle          14732
Dallas           14820
Atlanta          14881
Boston           19934
New York City    24876
Los Angeles      29605
San Francisco    44732
Name: Quantity Ordered, dtype: int64

In [6]:
df_ori.groupby('Product')['Quantity Ordered'].count().sort_values()

Product
LG Dryer                        646
LG Washing Machine              666
Vareebadd Phone                2065
20in Monitor                   4101
ThinkPad Laptop                4128
Macbook Pro Laptop             4724
Flatscreen TV                  4800
Google Phone                   5525
34in Ultrawide Monitor         6181
27in 4K Gaming Monitor         6230
iPhone                         6842
27in FHD Monitor               7507
Bose SoundSport Headphones    13325
Apple Airpods Headphones      15549
Wired Headphones              18882
AA Batteries (4-pack)         20577
AAA Batteries (4-pack)        20641
Lightning Charging Cable      21658
USB-C Charging Cable          21903
Name: Quantity Ordered, dtype: int64

In [7]:
df_ori.groupby('Product')['Quantity Ordered'].sum().sort_values()

Product
LG Dryer                        646
LG Washing Machine              666
Vareebadd Phone                2068
20in Monitor                   4129
ThinkPad Laptop                4130
Macbook Pro Laptop             4728
Flatscreen TV                  4819
Google Phone                   5532
34in Ultrawide Monitor         6199
27in 4K Gaming Monitor         6244
iPhone                         6849
27in FHD Monitor               7550
Bose SoundSport Headphones    13457
Apple Airpods Headphones      15661
Wired Headphones              20557
Lightning Charging Cable      23217
USB-C Charging Cable          23975
AA Batteries (4-pack)         27635
AAA Batteries (4-pack)        31017
Name: Quantity Ordered, dtype: int64

In [8]:
df_ori.query('City == " Atlanta"').groupby('Product')['Quantity Ordered'].sum().sort_values()

Product
LG Washing Machine              52
LG Dryer                        59
Vareebadd Phone                173
20in Monitor                   342
ThinkPad Laptop                357
Macbook Pro Laptop             379
Flatscreen TV                  407
Google Phone                   451
34in Ultrawide Monitor         482
27in 4K Gaming Monitor         493
iPhone                         544
27in FHD Monitor               588
Bose SoundSport Headphones    1084
Apple Airpods Headphones      1266
Wired Headphones              1579
Lightning Charging Cable      1879
USB-C Charging Cable          1915
AA Batteries (4-pack)         2193
AAA Batteries (4-pack)        2359
Name: Quantity Ordered, dtype: int64

### quantidades: Austin=9905, Boston=19934, San Francisco=44732

In [6]:
# renomeando campos
df_aux = df_ori.rename(columns={
  'Order ID': 'id', 
  'Product' : 'produto', 
  'Quantity Ordered': 'quantidade', 
  'Price': 'preco', 
  'Order Date': 'data',
  'Time': 'hora',
  'Product Type': 'categoria', 
  'Purchase Address': 'endereco', 
  'City': 'cidade',
})

# criando campo horário
df_aux['horario'] = pd.to_datetime(df_aux['data'] + ' ' + df_aux['hora'], format='%d-%m-%Y %I:%M %p')

# diminuindo quantidade de registros
df_aux['cidade'] = df_aux['cidade'].str.strip()
# df_aux = df_aux.query('horario.dt.year == 2019 and cidade == "San Francisco"')[
df_aux = df_aux.query('horario.dt.year == 2019 and cidade == "Atlanta"')[
  ['horario', 'produto', 'quantidade', 'preco']
]

### ajustando os valores

In [7]:
df = df_aux.sort_values('horario')

# campo auxiliar para aleatórios
df['rnd'] = [random.randrange(100) for _ in df.index]

# transferir 25% das vendas dos meses 3, 4 e 5 para os meses 9, 10 e 11
df['horario'] = df['horario'].mask(
  (df['horario'] > pd.to_datetime('01-03-2019', format='%d-%m-%Y')) & 
  (df['horario'] < pd.to_datetime('01-06-2019', format='%d-%m-%Y')) & 
  (df['rnd'] < 25), 
  df['horario'] + pd.DateOffset(months=6))

# transferir 20% das vendas do mês 12 para o mês 1
df['horario'] = df['horario'].mask(
  (df['horario'] > pd.to_datetime('01-12-2019', format='%d-%m-%Y')) & 
  (df['rnd'] < 20), 
  df['horario'] + pd.DateOffset(months=-11))

# distribuir aleatóriamente os registros em 5 anos
df['rnd'] = [random.randrange(100) for _ in df.index]
df['horario'] = df['horario'].mask(
  (df.rnd < 25), 
  df['horario'] + pd.DateOffset(years=2))
df['horario'] = df['horario'].mask(
  (df.rnd >= 25) & 
  (df.rnd < 50),
  df['horario'] + pd.DateOffset(years=3))
df['horario'] = df['horario'].mask(
  (df.rnd >= 50) & 
  (df.rnd < 75),
  df['horario'] + pd.DateOffset(years=4))
df['horario'] = df['horario'].mask(
  (df.rnd >= 75),
  df['horario'] + pd.DateOffset(years=5))

# campo auxiliar de data
df['data'] = pd.to_datetime(df['horario']).dt.date

# distribuir os itens do pedido
df['nota'] = df['horario'].dt.strftime('%Y%m%d-%M').str.slice(stop=10)

# permitir cálculo com o preço
df['preco'] = df['preco'].str.replace(',', '').astype(float)
df['preco'] = df.apply(lambda row: row['preco'] * 10 if row['preco'] < 80 else row['preco'], axis=1)
df['preco'] = df['preco'].round(2)

# calcular aleatóriamente quantidade, mas valores menores tem peso maior
pop = [i + 1 for i in range(40)]
wei = [i * i + 1 for i in range(40, 0, -1)]
df['quantidade'] = random.choices(pop, wei, k=len(df.index))
df['quantidade'] = (df['quantidade'] / (df['quantidade'] * df['preco'] / 10000 + 1).astype(int)).astype(int)
df['total'] = df['quantidade'] * df['preco']
df['total'] = df['total'].round(2)

# renomear produtos
prod = {
  'LG Dryer': 'P19',
  'LG Washing Machine': 'P20',
  'Vareebadd Phone': 'P21',
  '20in Monitor': 'P22',
  'ThinkPad Laptop': 'P23',
  'Macbook Pro Laptop': 'P24',
  'Flatscreen TV': 'P25',
  'Google Phone': 'P26',
  '34in Ultrawide Monitor': 'P27',
  '27in 4K Gaming Monitor': 'P28',
  'iPhone': 'P29',
  '27in FHD Monitor': '',
  'Bose SoundSport Headphones': '',
  'Apple Airpods Headphones': '',
  'Wired Headphones': '',
  'Lightning Charging Cable': '',
  'USB-C Charging Cable': '',
  'AA Batteries (4-pack': '',
  'AAA Batteries (4-pack': '',
}

df['produto'] = df.apply(
  lambda row: 
    prod[row['produto']] if row['produto'] in [
      'LG Dryer', 
      'LG Washing Machine', 
      'Vareebadd Phone', 
      '20in Monitor', 
      'ThinkPad Laptop', 
      'Macbook Pro Laptop', 
      'Flatscreen TV', 
      'Google Phone', 
      '34in Ultrawide Monitor', 
      '27in 4K Gaming Monitor', 
      'iPhone'
    ] else f'P{random.randrange(1, 19):02}', axis=1)

df = df[['nota', 'data', 'horario', 'produto', 'quantidade']].sort_values('horario')
df

,nota,data,horario,produto,quantidade
75699,20210101-0,2021-01-01,2021-01-01 06:03:00,P05,17
38786,20210101-2,2021-01-01,2021-01-01 09:28:00,P11,18
42231,20210101-3,2021-01-01,2021-01-01 10:31:00,P23,10
69829,20210101-1,2021-01-01,2021-01-01 11:16:00,P15,18
74381,20210101-2,2021-01-01,2021-01-01 11:25:00,P08,14
...,...,...,...,...,...
54406,20241231-0,2024-12-31,2024-12-31 18:00:00,P17,3
53704,20241231-5,2024-12-31,2024-12-31 18:56:00,P03,2
48703,20241231-4,2024-12-31,2024-12-31 19:45:00,P23,7
50386,20241231-5,2024-12-31,2024-12-31 19:57:00,P15,26


In [8]:
# df.head()
# df.info()
# df[:20]
# df[-20:]
df.shape
# df_aux.shape
# df_aux[:20]
# df_aux[-20:]

(14881, 5)

In [68]:
# quantidade por mês
j = df[['horario', 'quantidade']]
j.index = df.horario
j.resample(rule='ME', on='horario')['quantidade'].sum()

horario
2021-01-31    2793
2021-02-28    2203
2021-03-31    2398
2021-04-30    2544
2021-05-31    2152
2021-06-30    2746
2021-07-31    2913
2021-08-31    2364
2021-09-30    3291
2021-10-31    5306
2021-11-30    4401
2021-12-31    4239
2022-01-31    2993
2022-02-28    2291
2022-03-31    2219
2022-04-30    2793
2022-05-31    2496
2022-06-30    2949
2022-07-31    2874
2022-08-31    2426
2022-09-30    2744
2022-10-31    5098
2022-11-30    4799
2022-12-31    4076
2023-01-31    2922
2023-02-28    2682
2023-03-31    2253
2023-04-30    2525
2023-05-31    2503
2023-06-30    2787
2023-07-31    2666
2023-08-31    2295
2023-09-30    2985
2023-10-31    5002
2023-11-30    4302
2023-12-31    4132
2024-01-31    3102
2024-02-29    2348
2024-03-31    2272
2024-04-30    2467
2024-05-31    2546
2024-06-30    2913
2024-07-31    3034
2024-08-31    1981
2024-09-30    3020
2024-10-31    5244
2024-11-30    4480
2024-12-31    3854
Freq: ME, Name: quantidade, dtype: int64

# Preparar e Inserir Pedidos

In [9]:
q = f'horario < "{datetime.now().date()}"'
df_ins = df.query(q).sort_values(['data', 'nota', 'produto'])

In [10]:
df_ins[:30]
# df_ins[-30:]

,nota,data,horario,produto,quantidade
75699,20210101-0,2021-01-01,2021-01-01 06:03:00,P05,17
69829,20210101-1,2021-01-01,2021-01-01 11:16:00,P15,18
48319,20210101-1,2021-01-01,2021-01-01 13:15:00,P18,19
74381,20210101-2,2021-01-01,2021-01-01 11:25:00,P08,14
38786,20210101-2,2021-01-01,2021-01-01 09:28:00,P11,18
35607,20210101-2,2021-01-01,2021-01-01 14:21:00,P11,19
75181,20210101-3,2021-01-01,2021-01-01 18:31:00,P13,5
67645,20210101-3,2021-01-01,2021-01-01 14:36:00,P22,12
42231,20210101-3,2021-01-01,2021-01-01 10:31:00,P23,10
52665,20210101-4,2021-01-01,2021-01-01 21:42:00,P05,4


In [11]:
df_ins.shape

(12324, 5)

In [12]:
representantes = list(db['venda_representante'].find())

estoques = {}
for e in db['venda_estoque'].find():
  estoques[f"P{e['id']:02}"] = e

In [13]:
estoques

{'P01': {'_id': ObjectId('6642bf3dfa8ceb442718d263'),
  'id': 1,
  'produto': {'marca': {'nome': 'KazeKami'},
   'categoria': {'titulo': 'Secadores'},
   'descricao': 'Secador de Parede 1200W'},
  'quantidade': 995,
  'preco_compra': 149.73,
  'preco_venda': 299.46},
 'P02': {'_id': ObjectId('6642bf3dfa8ceb442718d264'),
  'id': 2,
  'produto': {'marca': {'nome': 'KazeKami'},
   'categoria': {'titulo': 'Secadores'},
   'descricao': 'Secador de Parede 1200W'},
  'quantidade': 994,
  'preco_compra': 169.9,
  'preco_venda': 339.8},
 'P03': {'_id': ObjectId('6642bf3dfa8ceb442718d265'),
  'id': 3,
  'produto': {'marca': {'nome': 'KazeKami'},
   'categoria': {'titulo': 'Secadores'},
   'descricao': 'Secador de Parede 1400W'},
  'quantidade': 1000,
  'preco_compra': 460.35,
  'preco_venda': 920.7},
 'P04': {'_id': ObjectId('6642bf3dfa8ceb442718d266'),
  'id': 4,
  'produto': {'marca': {'nome': 'KazeKami'},
   'categoria': {'titulo': 'Secadores'},
   'descricao': 'Secador de Parede 1400W (Preto

In [14]:
def gen_pedido(i, itens_pedido, total, representantes, horario):
  representante = random.choice(representantes)
  cliente = random.choice(representante['clientes'])

  etapas = [{
    'id': 1,
    'etapa': 'criado',
    'horario': horario - timedelta(seconds = random.randrange(9999, 99999)),
    'user_id': representante['id']
  }]
  
  etapa_id = 2
  if representante['id'] != 2:
    etapas.append({
      'id': etapa_id,
      'etapa': 'enviado',
      'horario': horario - timedelta(seconds = random.randrange(99, 9999)),
      'user_id': representante['id']
    })
    etapa_id += 1

  etapas.append({
    'id': etapa_id,
    'etapa': 'confirmado',
    'horario': horario,
    'user_id': 2
  })

  pedido = {
    'id': i,
    'etapa': 'confirmado',
    'etapas_pedido': etapas,
    'representante_id': representante['id'],
    'cliente': cliente,
    'horario': horario,
    'itens_pedido': itens_pedido,
    'total': round(total, 2)
  }
  return pedido


In [15]:
pedidos = []
itens_pedido = []
i = 0
nota = df_ins.iloc[0, 0]
horario = df_ins.iloc[0, 2]
prod_ant = ''
total = 0
for e in df_ins.itertuples():
  if nota != e.nota:
    i += 1
    pedido = gen_pedido(i, itens_pedido, total, representantes, horario)
    pedidos.append(pedido)
    itens_pedido = []
    prod_ant = ''
    total = 0
    nota = e.nota
    horario  = e.horario

  estoque = estoques[e.produto]
  total_prod = round(e.quantidade * estoque['preco_venda'], 2)
  total += total_prod
  if e.produto == prod_ant:
    itens_pedido[-1]['quantidade'] += e.quantidade
  else:
    item = {
      'id': estoque['id'],
      'produto': estoque['produto'],
      'preco_compra': estoque['preco_compra'],
      'preco_venda': estoque['preco_venda'],
      'quantidade': e.quantidade,
      'total': total_prod
    }
    itens_pedido.append(item)
    prod_ant = e.produto

if nota != df_ins.iloc[0, 0]:
  pedido = gen_pedido(i + 1, itens_pedido, total, representantes, horario)
  pedidos.append(pedido)

In [46]:
# from pprint import pprint
# pprint(pedidos)

# print(df_ins.iloc[0, 2])
# print(df_ins.iloc[0, 2] - timedelta(seconds = random.randrange(99, 9999)))
# print(df_ins.iloc[0, 2] - timedelta(seconds = random.randrange(9999, 99999)))
# print(df_ins.iloc[0, 2] - timedelta(seconds = random.randrange(99999, 999999)))

# timestamp = 1629845689
# data_hora = datetime.fromtimestamp(timestamp)
# data_hora


In [67]:
db['venda_pedido'].delete_many({})
db['venda_pedido'].insert_many(pedidos)

InsertManyResult([ObjectId('6643f1381b3d20379772847f'), ObjectId('6643f1381b3d203797728480'), ObjectId('6643f1381b3d203797728481'), ObjectId('6643f1381b3d203797728482'), ObjectId('6643f1381b3d203797728483'), ObjectId('6643f1381b3d203797728484'), ObjectId('6643f1381b3d203797728485'), ObjectId('6643f1381b3d203797728486'), ObjectId('6643f1381b3d203797728487'), ObjectId('6643f1381b3d203797728488'), ObjectId('6643f1381b3d203797728489'), ObjectId('6643f1381b3d20379772848a'), ObjectId('6643f1381b3d20379772848b'), ObjectId('6643f1381b3d20379772848c'), ObjectId('6643f1381b3d20379772848d'), ObjectId('6643f1381b3d20379772848e'), ObjectId('6643f1381b3d20379772848f'), ObjectId('6643f1381b3d203797728490'), ObjectId('6643f1381b3d203797728491'), ObjectId('6643f1381b3d203797728492'), ObjectId('6643f1381b3d203797728493'), ObjectId('6643f1381b3d203797728494'), ObjectId('6643f1381b3d203797728495'), ObjectId('6643f1381b3d203797728496'), ObjectId('6643f1381b3d203797728497'), ObjectId('6643f1381b3d2037977284

In [68]:
# atualizando controle do sequencial de pedidos
print(db['__schema__'].find_one({'name': 'venda_pedido'})['auto']['seq'])
print(len(pedidos))

db['__schema__'].update_one({'name': 'venda_pedido'}, {
  '$set': {
    'auto.seq': len(pedidos)
  }}, upsert=False)

print(db['__schema__'].find_one({'name': 'venda_pedido'}))

5862
5886
{'_id': ObjectId('6636a5166034bd29b7dcf81f'), 'name': 'venda_pedido', 'auto': {'field_names': ['id'], 'seq': 5886}, 'fields': {'cliente': {'type_code': 'object'}, 'etapa': {'type_code': 'string'}, 'etapas_pedido': {'type_code': 'array'}, 'horario': {'type_code': 'date'}, 'id': {'type_code': 'int'}, 'itens_pedido': {'type_code': 'array'}, 'representante_id': {'type_code': 'int'}, 'total': {'type_code': 'double'}}}


In [ ]:
# mantendo a diferença com 1000 no estoque
for k in estoques.keys():
  estoques[k]['quant_venda'] = 0

for p in pedidos:
  for i in p['itens_pedido']:
    estoques[f'P{i["id"]:02}']['quant_venda'] += i['quantidade']

for k in estoques.keys():
  q = estoques[k]['quant_venda']

  db['venda_estoque'].update_one({'id': estoques[k]['id']}, {
    '$set': {
      'quantidade': (q // 1000 + 1) * 1000 - q
    }}, upsert=False)

In [48]:
for e in db['venda_estoque'].find():
  print(e)

{'_id': ObjectId('6642bf3dfa8ceb442718d263'), 'id': 1, 'produto': {'marca': {'nome': 'KazeKami'}, 'categoria': {'titulo': 'Secadores'}, 'descricao': 'Secador de Parede 1200W'}, 'quantidade': 599, 'preco_compra': 149.73, 'preco_venda': 299.46}
{'_id': ObjectId('6642bf3dfa8ceb442718d264'), 'id': 2, 'produto': {'marca': {'nome': 'KazeKami'}, 'categoria': {'titulo': 'Secadores'}, 'descricao': 'Secador de Parede 1200W'}, 'quantidade': 637, 'preco_compra': 169.9, 'preco_venda': 339.8}
{'_id': ObjectId('6642bf3dfa8ceb442718d265'), 'id': 3, 'produto': {'marca': {'nome': 'KazeKami'}, 'categoria': {'titulo': 'Secadores'}, 'descricao': 'Secador de Parede 1400W'}, 'quantidade': 369, 'preco_compra': 460.35, 'preco_venda': 920.7}
{'_id': ObjectId('6642bf3dfa8ceb442718d266'), 'id': 4, 'produto': {'marca': {'nome': 'KazeKami'}, 'categoria': {'titulo': 'Secadores'}, 'descricao': 'Secador de Parede 1400W (Preto)'}, 'quantidade': 301, 'preco_compra': 469.26, 'preco_venda': 938.52}
{'_id': ObjectId('6642b